In [1]:
from google.colab import drive  # 드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 필요 라이브러리 import 
import numpy as np
import torch
import os
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision
import time
import copy
%matplotlib inline  
%config InlineBackend.figure_format='retina'
print ("PyTorch version:[%s]."%(torch.__version__))

# Device Configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("This notebook use [%s]."%(device))

PyTorch version:[1.8.1+cu101].
This notebook use [cuda:0].


In [3]:
EPOCHS =100
BATCH_SIZE= 32

In [4]:
from torchvision import datasets, transforms, models


transforms = transforms.Compose([transforms.ToTensor(),
                                 transforms.Resize((224,224)),
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                      [0.229, 0.224, 0.225])])





DATASET_PATH = "/content/drive/MyDrive/생체인증보안/multi"

#face dataset 불러오기
train_Fdata = datasets.ImageFolder(DATASET_PATH + '/train/face', transform=transforms)
val_Fdata = datasets.ImageFolder(DATASET_PATH + '/validate/face', transform=transforms)

train_Fiter = torch.utils.data.DataLoader(train_Fdata, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
val_Fiter = torch.utils.data.DataLoader(val_Fdata, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)

#iris dataset불러오기 
train_Idata = datasets.ImageFolder(DATASET_PATH + '/train/iris', transform=transforms)
val_Idata = datasets.ImageFolder(DATASET_PATH + '/validate/iris', transform=transforms)

train_Iiter = torch.utils.data.DataLoader(train_Idata, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
val_Iiter = torch.utils.data.DataLoader(val_Idata, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)


In [17]:
# face 모델
face_model = models.resnet18(pretrained=True)
num_ftrs = face_model.fc.in_features
face_model.fc = nn.Linear(num_ftrs, 64) 

face_model = face_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(face_model.parameters(), lr=0.001)

In [6]:
# iris 모델
iris_model = models.resnet18(pretrained=True)
num_ftrs = iris_model.fc.in_features
iris_model.fc = nn.Linear(num_ftrs, 64) 

iris_model = iris_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(iris_model.parameters(), lr=0.01)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [7]:
def test_eval(model, data_iter, batch_size):
    with torch.no_grad():
        test_loss = 0
        total = 0
        correct = 0
        model.eval()
        for batch_img, batch_lab in data_iter:
            X = batch_img.to(device)
            Y = batch_lab.to(device)
            y_pred = model(X)
            _, predicted = torch.max(y_pred.data, 1)
            correct += (predicted == Y).sum().item()
            total += batch_img.size(0)
        val_acc = (100 * correct / total)
        model.train()
    return val_acc

#Train

In [9]:
# 홍채 학습
print_every = 1
print("Start training ! (Iris)")

# Training loop
for epoch in range(EPOCHS):
    loss_val_sum = 0
    for batch_img, batch_lab in train_Iiter:

        X = batch_img.to(device)
        Y = batch_lab.to(device)
        
        y_pred = iris_model.forward(X)
        loss = criterion(y_pred, Y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_val_sum += loss
        
    if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
        loss_val_avg = loss_val_sum / len(train_Iiter)
        accr_val = test_eval(iris_model, val_Iiter, BATCH_SIZE)
        print(f"epoch:[{epoch+1}/{EPOCHS}] cost:[{loss_val_avg:.3f}] test_accuracy:[{accr_val:.3f}]")

print("Training Done !")

Start training ! (Iris)
epoch:[1/100] cost:[4.560] test_accuracy:[2.344]
epoch:[2/100] cost:[3.979] test_accuracy:[3.906]
epoch:[3/100] cost:[3.709] test_accuracy:[6.250]
epoch:[4/100] cost:[3.451] test_accuracy:[10.156]
epoch:[5/100] cost:[3.166] test_accuracy:[18.750]
epoch:[6/100] cost:[2.664] test_accuracy:[13.281]
epoch:[7/100] cost:[2.334] test_accuracy:[22.656]
epoch:[8/100] cost:[1.866] test_accuracy:[29.688]
epoch:[9/100] cost:[1.544] test_accuracy:[28.125]
epoch:[10/100] cost:[1.488] test_accuracy:[41.406]
epoch:[11/100] cost:[1.000] test_accuracy:[28.125]
epoch:[12/100] cost:[0.739] test_accuracy:[36.719]
epoch:[13/100] cost:[0.615] test_accuracy:[57.031]
epoch:[14/100] cost:[0.421] test_accuracy:[90.625]
epoch:[15/100] cost:[0.354] test_accuracy:[32.812]
epoch:[16/100] cost:[0.394] test_accuracy:[89.844]
epoch:[17/100] cost:[0.213] test_accuracy:[47.656]
epoch:[18/100] cost:[0.157] test_accuracy:[77.344]
epoch:[19/100] cost:[0.157] test_accuracy:[92.188]
epoch:[20/100] cost

In [20]:
# 얼굴 학습


print_every = 1
print("Start training ! (Face)")

EPOCHS_F = 30
# Training loop
for epoch in range(EPOCHS_F):
    loss_val_sum = 0
    for batch_img, batch_lab in train_Fiter:

        X = batch_img.to(device)
        Y = batch_lab.to(device)
        
        y_pred = face_model.forward(X)
        loss = criterion(y_pred, Y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_val_sum += loss
        
    if ((epoch%print_every)==0) or (epoch==(EPOCHS_F-1)):
        loss_val_avg = loss_val_sum / len(train_Fiter)
        accr_val = test_eval(face_model, val_Fiter, BATCH_SIZE)
        print(f"epoch:[{epoch+1}/{EPOCHS_F}] cost:[{loss_val_avg:.3f}] test_accuracy:[{accr_val:.3f}]")
print("Training Done !")

Start training ! (Face)
epoch:[1/30] cost:[0.000] test_accuracy:[100.000]
epoch:[2/30] cost:[0.000] test_accuracy:[100.000]
epoch:[3/30] cost:[0.000] test_accuracy:[100.000]
epoch:[4/30] cost:[0.000] test_accuracy:[100.000]
epoch:[5/30] cost:[0.000] test_accuracy:[100.000]
epoch:[6/30] cost:[0.000] test_accuracy:[100.000]
epoch:[7/30] cost:[0.000] test_accuracy:[100.000]
epoch:[8/30] cost:[0.000] test_accuracy:[100.000]
epoch:[9/30] cost:[0.000] test_accuracy:[100.000]
epoch:[10/30] cost:[0.000] test_accuracy:[100.000]
epoch:[11/30] cost:[0.000] test_accuracy:[100.000]
epoch:[12/30] cost:[0.000] test_accuracy:[100.000]
epoch:[13/30] cost:[0.000] test_accuracy:[100.000]
epoch:[14/30] cost:[0.000] test_accuracy:[100.000]
epoch:[15/30] cost:[0.000] test_accuracy:[100.000]
epoch:[16/30] cost:[0.000] test_accuracy:[100.000]
epoch:[17/30] cost:[0.000] test_accuracy:[100.000]
epoch:[18/30] cost:[0.000] test_accuracy:[100.000]
epoch:[19/30] cost:[0.000] test_accuracy:[100.000]
epoch:[20/30] co

#Validation

###iris validation

In [21]:
val_Iiter = torch.utils.data.DataLoader(val_Idata, batch_size=128, shuffle=True, num_workers=1)
data_Iiter = iter(val_Iiter)
images, labels = next(data_Iiter)

In [22]:
inputs, classes = next(iter(train_Iiter))
out = torchvision.utils.make_grid(inputs)

In [23]:
n_sample = 128
test_x = images[:n_sample]
test_y = labels[:n_sample]

with torch.no_grad():
    iris_model.eval()
    y_pred = iris_model.forward(test_x.type(torch.float).to(device))
    iris_model.train()
    
y_pred = y_pred.argmax(axis=1)

test_accuracy = sum([1 for i, j in zip(y_pred, test_y) if i == j]) / n_sample
print(f"test_accuracy : {test_accuracy:.3f}")

for idx in range(n_sample):
    print("Predict:",str(y_pred[idx]+1)[7:10], "Label:",str(test_y[idx]+1)[7:10].replace(')', ''))

test_accuracy : 1.000
Predict: 50, Label: 50
Predict: 13, Label: 13
Predict: 1,  Label: 1
Predict: 47, Label: 47
Predict: 46, Label: 46
Predict: 26, Label: 26
Predict: 2,  Label: 2
Predict: 34, Label: 34
Predict: 64, Label: 64
Predict: 51, Label: 51
Predict: 48, Label: 48
Predict: 22, Label: 22
Predict: 38, Label: 38
Predict: 3,  Label: 3
Predict: 18, Label: 18
Predict: 3,  Label: 3
Predict: 15, Label: 15
Predict: 6,  Label: 6
Predict: 44, Label: 44
Predict: 16, Label: 16
Predict: 34, Label: 34
Predict: 25, Label: 25
Predict: 41, Label: 41
Predict: 28, Label: 28
Predict: 61, Label: 61
Predict: 17, Label: 17
Predict: 28, Label: 28
Predict: 20, Label: 20
Predict: 56, Label: 56
Predict: 56, Label: 56
Predict: 26, Label: 26
Predict: 50, Label: 50
Predict: 18, Label: 18
Predict: 44, Label: 44
Predict: 47, Label: 47
Predict: 40, Label: 40
Predict: 43, Label: 43
Predict: 21, Label: 21
Predict: 10, Label: 10
Predict: 46, Label: 46
Predict: 4,  Label: 4
Predict: 54, Label: 54
Predict: 20, Label

###face validation 

In [24]:
val_Fiter = torch.utils.data.DataLoader(val_Fdata, batch_size=128, shuffle=True, num_workers=1)
data_Fiter = iter(val_Fiter)
images, labels = next(data_Fiter)

In [25]:
inputs, classes = next(iter(train_Fiter))
out = torchvision.utils.make_grid(inputs)

In [26]:
n_sample = 128
test_x = images[:n_sample]
test_y = labels[:n_sample]

with torch.no_grad():
    face_model.eval()
    y_pred = face_model.forward(test_x.type(torch.float).to(device))
    face_model.train()
    
y_pred = y_pred.argmax(axis=1)

test_accuracy = sum([1 for i, j in zip(y_pred, test_y) if i == j]) / n_sample
print(f"test_accuracy : {test_accuracy:.3f}")

for idx in range(n_sample):
    print("Predict:",str(y_pred[idx]+1)[7:10].replace(')', ''), "Label:",str(test_y[idx]+1)[7:10].replace(')', ''))

test_accuracy : 1.000
Predict: 55, Label: 55
Predict: 5,  Label: 5
Predict: 59, Label: 59
Predict: 24, Label: 24
Predict: 43, Label: 43
Predict: 32, Label: 32
Predict: 6,  Label: 6
Predict: 50, Label: 50
Predict: 39, Label: 39
Predict: 5,  Label: 5
Predict: 24, Label: 24
Predict: 60, Label: 60
Predict: 47, Label: 47
Predict: 46, Label: 46
Predict: 33, Label: 33
Predict: 54, Label: 54
Predict: 42, Label: 42
Predict: 27, Label: 27
Predict: 34, Label: 34
Predict: 22, Label: 22
Predict: 22, Label: 22
Predict: 4,  Label: 4
Predict: 28, Label: 28
Predict: 35, Label: 35
Predict: 36, Label: 36
Predict: 7,  Label: 7
Predict: 44, Label: 44
Predict: 7,  Label: 7
Predict: 20, Label: 20
Predict: 2,  Label: 2
Predict: 3,  Label: 3
Predict: 11, Label: 11
Predict: 21, Label: 21
Predict: 23, Label: 23
Predict: 16, Label: 16
Predict: 37, Label: 37
Predict: 38, Label: 38
Predict: 31, Label: 31
Predict: 58, Label: 58
Predict: 25, Label: 25
Predict: 62, Label: 62
Predict: 31, Label: 31
Predict: 9,  Label: 

#Test

###iris test

In [27]:
test_data = datasets.ImageFolder(DATASET_PATH + '/test/iris', transform=transforms)
test_iter = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=1)

class_name = test_data.classes
class_len = len(class_name)

In [28]:
data_iter = iter(test_iter)
images, labels = next(data_iter)

# 라벨링할 class는 train_iter에서
inputs, classes = next(iter(train_Iiter))
out = torchvision.utils.make_grid(inputs)

n_sample = 64
test_x = images[:n_sample]
test_y = labels[:n_sample]

with torch.no_grad():
    iris_model.eval()
    y_pred = iris_model.forward(test_x.type(torch.float).to(device))
    iris_model.train()
    
y_pred = y_pred.argmax(axis=1)

result_iris = []

for idx in range(n_sample):
    print( "File name:",str(class_name[test_y[idx]]),"Predict:",str(y_pred[idx])[7:10].replace(')',''))
    result_iris.append({'Iris image':str(class_name[test_y[idx]]), 
                        'Iris answer':str(y_pred[idx])[7:10].replace(')','')})



Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa4277557a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa4277557a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

File name: 0 Predict: 27,
File name: 1 Predict: 60,
File name: 10 Predict: 36,
File name: 11 Predict: 2, 
File name: 12 Predict: 46,
File name: 13 Predict: 1, 
File name: 14 Predict: 10,
File name: 15 Predict: 61,
File name: 16 Predict: 49,
File name: 17 Predict: 59,
File name: 18 Predict: 11,
File name: 19 Predict: 13,
File name: 2 Predict: 22,
File name: 20 Predict: 37,
File name: 21 Predict: 30,
File name: 22 Predict: 25,
File name: 23 Predict: 0, 
File name: 24 Predict: 11,
File name: 25 Predict: 5, 
File name: 26 Predict: 9, 
File name: 27 Predict: 20,
File name: 28 Predict: 48,
File name: 29 Predict: 47,
File name: 3 Predict: 57,
File name: 30 Predict: 42,
File name: 31 Predict: 24,
File name: 32 Predict: 3, 
File name: 33 Predict: 6, 
File name: 34 Predict: 55,
File name: 35 Predict: 63,
File name: 36 Predict: 21,
File name: 37 Predict: 16,
File name: 38 Predict: 34,
File name: 39 Predict: 11,
File name: 4 Predict: 54,
File name: 40 Predict: 35,
File name: 41 Predict: 43,
File n

#Test - Face

In [29]:
test_data = datasets.ImageFolder(DATASET_PATH + '/test/face', transform=transforms)
test_iter = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=1)

class_name = test_data.classes
class_len = len(class_name)

In [30]:
data_iter = iter(test_iter)
images, labels = next(data_iter)

# 라벨링할 class는 train_iter에서
inputs, classes = next(iter(train_Fiter))
out = torchvision.utils.make_grid(inputs)

n_sample = 64
test_x = images[:n_sample]
test_y = labels[:n_sample]

with torch.no_grad():
    face_model.eval()
    y_pred = face_model.forward(test_x.type(torch.float).to(device))
    face_model.train()
    
y_pred = y_pred.argmax(axis=1)

result_face = []

for idx in range(n_sample):
    print( "File name:",str(class_name[test_y[idx]]),"Predict:",str(y_pred[idx])[7:10])
    result_face.append({'Face image':str(class_name[test_y[idx]]), 
                        'Face answer':str(y_pred[idx])[7:10].replace(')','')})


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa4277557a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa4277557a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

File name: 0 Predict: 27,
File name: 1 Predict: 31,
File name: 10 Predict: 36,
File name: 11 Predict: 53,
File name: 12 Predict: 46,
File name: 13 Predict: 50,
File name: 14 Predict: 10,
File name: 15 Predict: 61,
File name: 16 Predict: 49,
File name: 17 Predict: 59,
File name: 18 Predict: 56,
File name: 19 Predict: 13,
File name: 2 Predict: 22,
File name: 20 Predict: 37,
File name: 21 Predict: 30,
File name: 22 Predict: 25,
File name: 23 Predict: 19,
File name: 24 Predict: 41,
File name: 25 Predict: 58,
File name: 26 Predict: 9, 
File name: 27 Predict: 20,
File name: 28 Predict: 48,
File name: 29 Predict: 47,
File name: 3 Predict: 57,
File name: 30 Predict: 42,
File name: 31 Predict: 24,
File name: 32 Predict: 3, 
File name: 33 Predict: 6, 
File name: 34 Predict: 55,
File name: 35 Predict: 63,
File name: 36 Predict: 21,
File name: 37 Predict: 16,
File name: 38 Predict: 34,
File name: 39 Predict: 11,
File name: 4 Predict: 54,
File name: 40 Predict: 35,
File name: 41 Predict: 32,
File n

In [31]:
result_face

[{'Face answer': '27,', 'Face image': '0'},
 {'Face answer': '31,', 'Face image': '1'},
 {'Face answer': '36,', 'Face image': '10'},
 {'Face answer': '53,', 'Face image': '11'},
 {'Face answer': '46,', 'Face image': '12'},
 {'Face answer': '50,', 'Face image': '13'},
 {'Face answer': '10,', 'Face image': '14'},
 {'Face answer': '61,', 'Face image': '15'},
 {'Face answer': '49,', 'Face image': '16'},
 {'Face answer': '59,', 'Face image': '17'},
 {'Face answer': '56,', 'Face image': '18'},
 {'Face answer': '13,', 'Face image': '19'},
 {'Face answer': '22,', 'Face image': '2'},
 {'Face answer': '37,', 'Face image': '20'},
 {'Face answer': '30,', 'Face image': '21'},
 {'Face answer': '25,', 'Face image': '22'},
 {'Face answer': '19,', 'Face image': '23'},
 {'Face answer': '41,', 'Face image': '24'},
 {'Face answer': '58,', 'Face image': '25'},
 {'Face answer': '9, ', 'Face image': '26'},
 {'Face answer': '20,', 'Face image': '27'},
 {'Face answer': '48,', 'Face image': '28'},
 {'Face answe

In [32]:
result_iris

[{'Iris answer': '27,', 'Iris image': '0'},
 {'Iris answer': '60,', 'Iris image': '1'},
 {'Iris answer': '36,', 'Iris image': '10'},
 {'Iris answer': '2, ', 'Iris image': '11'},
 {'Iris answer': '46,', 'Iris image': '12'},
 {'Iris answer': '1, ', 'Iris image': '13'},
 {'Iris answer': '10,', 'Iris image': '14'},
 {'Iris answer': '61,', 'Iris image': '15'},
 {'Iris answer': '49,', 'Iris image': '16'},
 {'Iris answer': '59,', 'Iris image': '17'},
 {'Iris answer': '11,', 'Iris image': '18'},
 {'Iris answer': '13,', 'Iris image': '19'},
 {'Iris answer': '22,', 'Iris image': '2'},
 {'Iris answer': '37,', 'Iris image': '20'},
 {'Iris answer': '30,', 'Iris image': '21'},
 {'Iris answer': '25,', 'Iris image': '22'},
 {'Iris answer': '0, ', 'Iris image': '23'},
 {'Iris answer': '11,', 'Iris image': '24'},
 {'Iris answer': '5, ', 'Iris image': '25'},
 {'Iris answer': '9, ', 'Iris image': '26'},
 {'Iris answer': '20,', 'Iris image': '27'},
 {'Iris answer': '48,', 'Iris image': '28'},
 {'Iris answe

In [38]:
result = []

for i in range(64):
  iris_pred = result_iris[i].get('Iris answer')
  face_pred = result_face[i].get('Face answer')
  label = result_iris[i].get('Iris image')
  print(label)
  print("iris pred : ",iris_pred, "  face pred : ",face_pred)
  if (iris_pred != face_pred):
    result.append({'image':label,'answer':iris_pred})
  else :
    result.append({'image':label,'answer':iris_pred})

0
iris pred :  27,   face pred :  27,
1
iris pred :  60,   face pred :  31,
10
iris pred :  36,   face pred :  36,
11
iris pred :  2,    face pred :  53,
12
iris pred :  46,   face pred :  46,
13
iris pred :  1,    face pred :  50,
14
iris pred :  10,   face pred :  10,
15
iris pred :  61,   face pred :  61,
16
iris pred :  49,   face pred :  49,
17
iris pred :  59,   face pred :  59,
18
iris pred :  11,   face pred :  56,
19
iris pred :  13,   face pred :  13,
2
iris pred :  22,   face pred :  22,
20
iris pred :  37,   face pred :  37,
21
iris pred :  30,   face pred :  30,
22
iris pred :  25,   face pred :  25,
23
iris pred :  0,    face pred :  19,
24
iris pred :  11,   face pred :  41,
25
iris pred :  5,    face pred :  58,
26
iris pred :  9,    face pred :  9, 
27
iris pred :  20,   face pred :  20,
28
iris pred :  48,   face pred :  48,
29
iris pred :  47,   face pred :  47,
3
iris pred :  57,   face pred :  57,
30
iris pred :  42,   face pred :  42,
31
iris pred :  24,   face pr

In [39]:
from openpyxl import Workbook # 값을 자동으로 엑셀에 저장하는 코드
 
wb = Workbook()

sheet1 = wb.active

sheet1.title = 'sampleSheet'
fname = 'result_멀티모달2개_2차.xlsx'

n = len(result)

for i in range(n):
  sheet1.cell(row=i+1, column=1).value = result[i].get('image')
  sheet1.cell(row=i+1, column=2).value =  str(result[i].get('answer')).replace(',','')

wb.save(filename=fname)